In [1]:
import datajoint as dj
import numpy as np

In [2]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
ndap100 = dj.create_virtual_module("ndap100","microns_ndap100")

Connecting celiib@10.28.0.34:3306


In [3]:
ta3p100.FilteredSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,n_edges,edges,lengths_of_removed_limbs
2,648518346341351467,3000,517,=BLOB=,=BLOB=
2,648518346341351503,3000,323,=BLOB=,=BLOB=
2,648518346341351508,3000,486,=BLOB=,=BLOB=
2,648518346341351509,3000,348,=BLOB=,=BLOB=
2,648518346341351512,3000,576,=BLOB=,=BLOB=
2,648518346341351514,3000,558,=BLOB=,=BLOB=
2,648518346341351518,3000,391,=BLOB=,=BLOB=


In [6]:
ta3p100.NeuriteRawSkeleton().proj() & ta3p100.NeuronRawSkeleton().proj()

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated


In [9]:
#will get the number of segments with 0 edges, the only soma ones will not be included in this list because this is before the soma filtering

trace_rel = dj.U('segmentation', 'segment_id') & ndap100.Trace.proj() & ta3p100.CurrentSegmentation
full_rel = trace_rel + ta3p100.AllenSoma.proj()
all_erro_seg = ((ta3p100.NeuronRawSkeleton & full_rel) & 'n_edges=0') - ta3p100.FilteredSkeletonMinusSoma().proj()

#ones that checked that were error segments:
"""
648518346341376439
648518346342803868
648518346348204225

"""



'\n648518346341376439\n648518346342803868\n648518346348204225\n\n'

In [12]:
#add the segments to segment exclude: 
segment_ids = all_erro_seg.fetch("segment_id")
segment_ids

array([648518346341353986, 648518346341361714, 648518346341369389,
       648518346341375544, 648518346341377105, 648518346341384074,
       648518346342798306, 648518346342802638, 648518346348190817,
       648518346348213113, 648518346349506558, 648518346341354006,
       648518346341387646, 648518346342803868, 648518346348204225,
       648518346349377026, 648518346349383777, 648518346341369691,
       648518346341379048, 648518346341384207, 648518346341388085,
       648518346341417143, 648518346342801263, 648518346342918331,
       648518346344149338, 648518346341397739, 648518346341403352,
       648518346342801266, 648518346345768511, 648518346348459912,
       648518346349377627, 648518346341358018, 648518346341403397,
       648518346342808593, 648518346349503678, 648518346349506591,
       648518346341356348, 648518346341377285, 648518346341420245,
       648518346342792927, 648518346342796312, 648518346346998356,
       648518346349484154, 648518346349492054, 648518346349506

In [14]:
#add the errored segment_ids to the exclusion table
ta3p100.SegmentExclude.insert([dict(segmentation=2,segment_id=si,criteria_id=6) for si in segment_ids],skip_duplicates=True)

In [15]:
# (ta3p100.SegmentExclude()) #was 237, now should be 351
(ta3p100.SegmentExclude()) #was 237, now should be 351

segmentation segmentation id,segment_id segment id unique within each Segmentation,criteria_id key by which to lookup the exclusion criteria description
2,648518346341353521,0
2,648518346341353986,0
2,648518346341354006,0
2,648518346341354370,0
2,648518346341355016,0
2,648518346341356348,0
2,648518346341356608,0


In [33]:
""" Check that are still in the table 
648518346341353521
648518346341353986



"""

ta3p100.SegmentExclude() & "segment_id=648518346341353986" #"criteria_id>5"

segmentation segmentation id,segment_id segment id unique within each Segmentation,criteria_id key by which to lookup the exclusion criteria description
2,648518346341353986,0
2,648518346341353986,7
2,648518346341353986,8
2,648518346341353986,9


In [8]:
#ta3p100.ExclusionCriteria.insert1(dict(criteria_id=6,criteria_desc="Entire Mesh Segmentation Error"))
#(ta3p100.ExclusionCriteria() & "criteria_id=6").delete()
ta3p100.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,merge errors
1,functional errors
2,multiple soma merge errors
3,merge errors from jcotton81
4,merge errors from jake and cameron
5,ori or sta discrepancy from paul
6,Entire Mesh Segmentation Error


In [7]:
(dj.U("segment_id") & ta3p100.NeuriteRawSkeleton().proj()) & (dj.U("segment_id") & ta3p100.NeuronRawSkeleton().proj())

segment_id segment id unique within each Segmentation
648518346341353986
648518346341355016
648518346341366545
648518346341368392
648518346341368825
648518346341394999
648518346341403352


In [8]:
#in neurons
ta3p100.FilteredSkeleton()


segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,n_edges,edges,lengths_of_removed_limbs
2,648518346341351467,3000,517,=BLOB=,=BLOB=
2,648518346341351503,3000,323,=BLOB=,=BLOB=
2,648518346341351508,3000,486,=BLOB=,=BLOB=
2,648518346341351509,3000,348,=BLOB=,=BLOB=
2,648518346341351512,3000,576,=BLOB=,=BLOB=
2,648518346341351514,3000,558,=BLOB=,=BLOB=
2,648518346341351518,3000,391,=BLOB=,=BLOB=


In [16]:
ta3p100.NeuriteRawSkeleton() & "n_edges<20"

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,lagest_mesh_perc the percentage of the entire mesh that the largest submesh makes up


In [20]:
ta3p100.FilteredSkeletonMinusSoma()

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,vertices vertex coordinates of soma mesh,triangles faces for soma mesh,edges edges of skeelton after soma edges removed,n_edges number of edges of skeelton after soma edges removed,soma_bounding_corners bounding box corners for the soma mesh
2,648518346341353574,3000,=BLOB=,=BLOB=,=BLOB=,521,=BLOB=
2,648518346341366885,3000,=BLOB=,=BLOB=,=BLOB=,932,=BLOB=
2,648518346341369721,3000,=BLOB=,=BLOB=,=BLOB=,54,=BLOB=
2,648518346341371119,3000,=BLOB=,=BLOB=,=BLOB=,855,=BLOB=
2,648518346341373109,3000,=BLOB=,=BLOB=,=BLOB=,77,=BLOB=
2,648518346341375427,3000,=BLOB=,=BLOB=,=BLOB=,969,=BLOB=
2,648518346341376439,3000,=BLOB=,=BLOB=,=BLOB=,0,=BLOB=


# See if got all of the complete error segments in Segment Exclude

In [25]:
from collections import Counter
ta3p100.LabelKey.fetch()
a = [10, 7, 8, 10, 10, 10, 10]
myCounter = Counter(a)
myCounter[10]

5

In [30]:
from collections import Counter
from tqdm import tqdm

segment_ids,vertices = ta3p100.CoarseLabelOrphan.fetch("segment_id","vertices")
err_segments = []
for i,verts in tqdm(enumerate(vertices)):

    myCounter = Counter(verts)
    if len(verts) == myCounter[10]:
        err_segments.append(segment_ids[i])
    
    

702it [00:13, 12.10it/s]


In [32]:
len(err_segments)

112

In [33]:
from collections import Counter
from tqdm import tqdm

segment_ids,vertices = ta3p100.CoarseLabelFinal.fetch("segment_id","vertices")
err_segments_reg = []
for i,verts in tqdm(enumerate(vertices)):

    myCounter = Counter(verts)
    if len(verts) == myCounter[10]:
        err_segments_reg.append(segment_ids[i])
    
    

322it [00:27, 11.65it/s]


In [34]:
len(err_segments_reg)

2

In [37]:
#find if they are in segment exclude
SegExcludeErrors = ta3p100.SegmentExclude.fetch("segment_id")


for seg in (err_segments_reg + err_segments):
    if seg not in SegExcludeErrors:
        print(str(seg) + " not in segentExclude")
    